In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from sklearn.model_selection import train_test_split

#ambil dan konvert data
def convert (x):
  if x == 1: return [1, -1]
  else: return [-1, 1]

def masukan_data(data):
    temp=[]
    for x in range(len(data)):
        temp.append(data.values[x])
    return temp
def konvert_kelas(data):
    temp=masukan_data(data)
    konvert=[]
    for x in range(len(data)):
        konvert.append(convert (temp[x]))
    return konvert

#machine learning
def transpose(data):
    hasil_transpose=np.transpose(data)
    return hasil_transpose

def Menghitung_H_init(Data_Latih,W_transpose): # Menghitung inisialisasi output hidden layer
    H_init=np.matmul(Data_Latih,W_transpose)
    return H_init

def Menghitung_H (H_init): #H = 1/(1+exp(-H_init))
    H_exponen=np.exp(-H_init)
    H = []
    for x in range(len(H_init)):
        H.append([])
        for y in range(len(H_init[0])):
            temp = 1 / (1 + H_exponen[x][y])
            H[x].append(temp)
    return H
#     return 1 / (1+np.exp(-H_init))

def Menghitung_H_plus (H): #H+=(HT.H)-1.HT
    H_transpose = transpose(H)
    HTemp1 = np.matmul(H_transpose, H)
    Htemp2 = np.linalg.inv(HTemp1)
    H_Plus = np.matmul(Htemp2, H_transpose)
    return H_Plus
#     return np.linalg.inv(H.T.dot(H)).dot(H.T)

def Menghitung_B_Topi (H_plus,Kelas_Train):
    B_Topi = np.matmul(H_plus, Kelas_Train)
    return B_Topi

def Menghitung_Y_Estimasi(H,B_Topi):
    Y = np.matmul(H, B_Topi)
    return Y

def Menghitung_Prediksi(Y_test):
    Prediksi=[]
    for x in range(len(Y_test)):
        temp=Y_test[x][0]
        counter=1
        for y in range(len(Y_test[0])):
            if(Y_test[x][y]>temp):
                temp=Y_test[x][y]
                counter=y+1
        Prediksi.append(counter)
    return Prediksi #bisa pake argmax

def confusion_matrix(Estimasi,kelas_asli):
    confusion=[[0,0],
               [0,0]]
    for x in range(len(Estimasi)):
         if (Estimasi[x] == 1 and kelas_asli[x]==1):
                confusion[0][0]=confusion[0][0]+1
         elif (Estimasi[x] == 1 and kelas_asli[x]==2):
                confusion[0][1]=confusion[0][1]+1
         elif (Estimasi[x] == 2 and kelas_asli[x]==1):
                confusion[1][0]=confusion[1][0]+1
         else:
            confusion[1][1]=confusion[1][1]+1
    return confusion

def Evaluasi(Y_test,kelas_asli):
    precision=[]
    recall=[]
    Kelas_Prediksi= Menghitung_Prediksi(Y_test)
    #Confusion Matrix
    confusion = confusion_matrix (Kelas_Prediksi,kelas_asli)
    for x in range (len(confusion)):
        print (confusion[x])
    #Precision
    for x in range(len(Y_test[0])):
        temp1= (confusion[x][x]/(np.sum(confusion[x])))*100
        precision.append(temp1)
    #Recall
    for i in range (len(Y_test[0])):
        temp2=0
        for j in range (len(Y_test[0])):
            temp2=temp2 + confusion[j][i]
        temp2=(confusion[i][i]/temp2)*100
        recall.append(temp2)
    #Akurasi
    temp3=0
    for z in range (len(Y_test[0])):
        temp3=temp3+confusion[z][z]
    akurasi=(temp3/np.sum(confusion))*100
    return precision,recall,akurasi

In [2]:
# Standard
# Raw=pd.read_csv('Dataset/Data Telco Proses Raw V2.csv')
# p = len(Raw) - 1000
# X_train = Raw.iloc[:p]
# X_test = Raw.iloc[p:]
# X_train_kelas = X_train["Churn"]
# X_test_kelas = X_test["Churn"]
# X_train=X_train.drop(labels = ["Churn"],axis = 1)
# X_test=X_test.drop(labels = ["Churn"],axis = 1)
# kelas_training=np.asarray(konvert_kelas(X_train_kelas))
# kelas_test=np.asarray(konvert_kelas(X_test_kelas))
# train=np.asarray(masukan_data(X_train))
# test=np.asarray(masukan_data(X_test))

#Dasa testing diambil random
Raw=pd.read_csv('Dataset/Data Telco Proses Raw V1,75.csv')
X_train ,X_test = train_test_split(Raw,test_size=0.1422)
X_train_kelas = X_train["Churn"]
X_test_kelas = X_test["Churn"]
X_train=X_train.drop(labels = ["Churn"],axis = 1)
X_test=X_test.drop(labels = ["Churn"],axis = 1)
kelas_training=np.asarray(konvert_kelas(X_train_kelas))
kelas_test=np.asarray(konvert_kelas(X_test_kelas))
train=np.asarray(masukan_data(X_train))
test=np.asarray(masukan_data(X_test))

In [3]:
#Training
for x in range (5):
    w_JK=np.random.uniform(low=-1, high=1, size=(6,7))#untuk size yg pertama buat ke bawah yg 1 lg buat ke samping (jumlah neuron, banyak fitur)
    W_transpose= transpose(w_JK)
    H_init=Menghitung_H_init(train,W_transpose)
    H=Menghitung_H(H_init)
    H_plus=Menghitung_H_plus(H)
    B_topi=Menghitung_B_Topi(H_plus,kelas_training)

    #Testing
    kelas_test_asli=np.asarray(masukan_data(X_test_kelas))
    H_init_Test=Menghitung_H_init(test,W_transpose)
    H_Test=Menghitung_H(H_init_Test)
    Y_test=Menghitung_Y_Estimasi(H_Test,B_topi)
    Testing=Menghitung_Prediksi(Y_test)
    print ("================================================")
    Precision,Recall,Akurasi= Evaluasi(Y_test,kelas_test_asli)
    print("Precision Churn= ",Precision[0]," %")
    print("Precision non Churn= ",Precision[1]," %")
    print("Recall Churn = ",Recall[0]," %") 
    print("Recall non Churn = ",Recall[1]," %") 
    print("Akurasi   = ",Akurasi," %")
    print ("================================================")

[72, 36]
[213, 679]
Precision Churn=  66.66666666666666  %
Precision non Churn=  76.12107623318386  %
Recall Churn =  25.263157894736842  %
Recall non Churn =  94.96503496503496  %
Akurasi   =  75.1  %
[285, 715]
[0, 0]
Precision Churn=  28.499999999999996  %
Precision non Churn=  nan  %
Recall Churn =  100.0  %
Recall non Churn =  0.0  %
Akurasi   =  28.499999999999996  %
[83, 54]
[202, 661]
Precision Churn=  60.58394160583942  %
Precision non Churn=  76.59327925840093  %
Recall Churn =  29.122807017543863  %
Recall non Churn =  92.44755244755245  %
Akurasi   =  74.4  %

C:\Users\Daris\Miniconda3\lib\site-packages\ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in long_scalars



[90, 40]
[195, 675]
Precision Churn=  69.23076923076923  %
Precision non Churn=  77.58620689655173  %
Recall Churn =  31.57894736842105  %
Recall non Churn =  94.4055944055944  %
Akurasi   =  76.5  %
[45, 17]
[240, 698]
Precision Churn=  72.58064516129032  %
Precision non Churn=  74.4136460554371  %
Recall Churn =  15.789473684210526  %
Recall non Churn =  97.62237762237763  %
Akurasi   =  74.3  %
